In [ ]:
vec3 GetNormal()
{
   vec3 a = vec3(gl_in[0].gl_Position) - vec3(gl_in[1].gl_Position);
   vec3 b = vec3(gl_in[2].gl_Position) - vec3(gl_in[1].gl_Position);
   return normalize(cross(a, b));
}  

In [ ]:
vec4 explode(vec4 position, vec3 normal)
{
    float magnitude = 2.0;
    vec3 direction = normal * ((sin(time) + 1.0) / 2.0) * magnitude; 
    return position + vec4(direction, 0.0);
} 

## Vertex Shader

In [ ]:
#version 330 core
layout (location = 0) in vec3 aPos;
layout (location = 2) in vec2 aTexCoords;

out VS_OUT {
    vec2 texCoords;
} vs_out;

uniform mat4 projection;
uniform mat4 view;
uniform mat4 model;

void main()
{
    vs_out.texCoords = aTexCoords;
    gl_Position = projection * view * model * vec4(aPos, 1.0); 
}

## Fragment Shader

In [ ]:
#version 330 core
out vec4 FragColor;

in vec2 TexCoords;

uniform sampler2D texture_diffuse1;

void main()
{
    FragColor = texture(texture_diffuse1, TexCoords);
}

## Geometry Shader

In [ ]:
#version 330 core
layout (triangles) in;
layout (triangle_strip, max_vertices = 3) out;

in VS_OUT {
    vec2 texCoords;
} gs_in[];

out vec2 TexCoords; 

uniform float time;

vec4 explode(vec4 position, vec3 normal)
{
    float magnitude = 2.0;
    vec3 direction = normal * ((sin(time) + 1.0) / 2.0) * magnitude; 
    return position + vec4(direction, 0.0);
}

vec3 GetNormal()
{
    vec3 a = vec3(gl_in[0].gl_Position) - vec3(gl_in[1].gl_Position);
    vec3 b = vec3(gl_in[2].gl_Position) - vec3(gl_in[1].gl_Position);
    return normalize(cross(a, b));
}

void main() {    
    vec3 normal = GetNormal();

    gl_Position = explode(gl_in[0].gl_Position, normal);
    TexCoords = gs_in[0].texCoords;
    EmitVertex();
    gl_Position = explode(gl_in[1].gl_Position, normal);
    TexCoords = gs_in[1].texCoords;
    EmitVertex();
    gl_Position = explode(gl_in[2].gl_Position, normal);
    TexCoords = gs_in[2].texCoords;
    EmitVertex();
    EndPrimitive();
}

# Exploding Bunny Code

In [1]:
#! /usr/bin/env python
# -*- coding: utf-8 -*-
""" Basic Object Manipulation Using the mouse and keyboard
"""

from __future__ import print_function

from OpenGL.GL import *
from OpenGL.GL.ARB import *
from OpenGL.GLU import *
from OpenGL.GLUT import *
from OpenGL.GLUT.special import *
from OpenGL.GL.shaders import *
from PIL.Image import open as pil_open


import glfw
import sys
import os
import glm
import time
import objloader
import math as mathf

# Global window
window = None
null = c_void_p(0)

ViewMatrix = glm.mat4(1.0)
ProjectionMatrix = glm.mat4(1.0)

def getViewMatrix():
    return ViewMatrix

def getProjectionMatrix():
    return ProjectionMatrix


# Initial position : on +Z
position = glm.vec3( 0.2, 0, 2 )
# Initial horizontal angle : toward -Z
horizontalAngle = 3.14
# Initial vertical angle : none
verticalAngle = 0.0
# Initial Field of View
initialFoV = 45.0

speed = 3.0 # 3 units / second
mouseSpeed = 0.002

lastTime = None

frame_count = 0

def pre_frame():
    pass
    
def post_fram():
    frame_count += 1

def disable_vsyc():
    import glfw
    glfw.swap_interval(0)

def enable_vsyc():
    import glfw
    glfw.swap_interval(1)

#return GLuint
def LoadShaders(vertex_file_path,fragment_file_path,geometry_file_path):
	# Create the shaders
    VertexShaderID = glCreateShader(GL_VERTEX_SHADER)
    FragmentShaderID = glCreateShader(GL_FRAGMENT_SHADER)
    GeometryShaderID = glCreateShader(GL_GEOMETRY_SHADER)

	# Read the Vertex Shader code from the file
    VertexShaderCode = ""
    with open(vertex_file_path,'r') as fr:
    	for line in fr:
            VertexShaderCode += line
		# alternatively you could use fr.readlines() and then join in to a single string

	# Read the Geometry Shader code from the file
    GeometryShaderCode = ""
    with open(geometry_file_path,'r') as fr:
        for line in fr:
            GeometryShaderCode += line
		# alternatively you could use fr.readlines() and then join in to a single string 

    FragmentShaderCode = ""
    with open(fragment_file_path,'r') as fr:
        for line in fr:
            FragmentShaderCode += line
		# alternatively you could use fr.readlines() and then join in to a single string 

	# Compile Vertex Shader
    print("Compiling shader: %s"%(vertex_file_path))
    glShaderSource(VertexShaderID, VertexShaderCode)
    glCompileShader(VertexShaderID)

	# Check Vertex Shader
    result = glGetShaderiv(VertexShaderID, GL_COMPILE_STATUS)
    if not result:
        raise RuntimeError(glGetShaderInfoLog(VertexShaderID))
        
    # Compile Geometry Shader
    print("Compiling shader: %s"%(geometry_file_path))
    glShaderSource(GeometryShaderID, GeometryShaderCode)
    glCompileShader(GeometryShaderID)

	# Check Geometry Shader
    result = glGetShaderiv(GeometryShaderID, GL_COMPILE_STATUS)
    if not result:
        raise RuntimeError(glGetShaderInfoLog(GeometryShaderID))

	# Compile Fragment Shader
    print("Compiling shader: %s"%(fragment_file_path))
    glShaderSource(FragmentShaderID,FragmentShaderCode)
    glCompileShader(FragmentShaderID)

	# Check Fragment Shader
    result = glGetShaderiv(VertexShaderID, GL_COMPILE_STATUS)
    if not result:
        raise RuntimeError(glGetShaderInfoLog(FragmentShaderID))



	# Link the program
    print("Linking program")
    ProgramID = glCreateProgram()
    glAttachShader(ProgramID, VertexShaderID)
    glAttachShader(ProgramID, FragmentShaderID)
    glAttachShader(ProgramID, GeometryShaderID)
    glLinkProgram(ProgramID)

	# Check the program
    result = glGetShaderiv(VertexShaderID, GL_COMPILE_STATUS)
    if not result:
        raise RuntimeError(glGetShaderInfoLog(ProgramID))

    glDeleteShader(VertexShaderID);
    glDeleteShader(FragmentShaderID);
    glDeleteShader(GeometryShaderID);

    return ProgramID;

def computeMatricesFromInputs(window):
    global lastTime
    global position
    global horizontalAngle
    global verticalAngle
    global initialFoV
    global ViewMatrix
    global ProjectionMatrix

    # glfwGetTime is called only once, the first time this function is called
    if lastTime == None:
        lastTime = glfw.get_time()

    # Compute time difference between current and last frame
    currentTime = glfw.get_time()
    deltaTime = currentTime - lastTime

    # Get mouse position
    xpos,ypos = glfw.get_cursor_pos(window)
    # Reset mouse position for next frame
    glfw.set_cursor_pos(window, 1024/2, 768/2);

    # Compute new orientation
    horizontalAngle +=mouseSpeed * float(1024.0/2.0-xpos);
    verticalAngle   += mouseSpeed * float(768.0/2.0-ypos);

    # Direction : Spherical coordinates to Cartesian coordinates conversion
    direction = glm.vec3(
        mathf.cos(verticalAngle) * mathf.sin(horizontalAngle), 
        mathf.sin(verticalAngle),
        mathf.cos(verticalAngle) * mathf.cos(horizontalAngle)
    )
    
    # Right vector
    right = glm.vec3(
        mathf.sin(horizontalAngle - 3.14/2.0), 
        0.0,
        mathf.cos(horizontalAngle - 3.14/2.0)
    )
    
    # Up vector
    up = glm.cross( right, direction )

    # Move forward
    if glfw.get_key( window, glfw.KEY_UP ) == glfw.PRESS or glfw.get_key( window, glfw.KEY_W ) == glfw.PRESS:
        position += direction * deltaTime * speed;
    
    # Move backward
    if glfw.get_key( window, glfw.KEY_DOWN ) == glfw.PRESS or glfw.get_key( window, glfw.KEY_S ) == glfw.PRESS:
        position -= direction * deltaTime * speed
    
    # Strafe right

    if glfw.get_key( window, glfw.KEY_RIGHT ) == glfw.PRESS or glfw.get_key( window, glfw.KEY_D ) == glfw.PRESS:
        position += right * deltaTime * speed
    
    # Strafe left
    if glfw.get_key( window, glfw.KEY_LEFT ) == glfw.PRESS or glfw.get_key( window, glfw.KEY_A ) == glfw.PRESS:
        position -= right * deltaTime * speed
    
    FoV = initialFoV #- 5 * glfwGetMouseWheel(); # Now GLFW 3 requires setting up a callback for this. It's a bit too complicated for this beginner's tutorial, so it's disabled instead.

    # Projection matrix : 45 Field of View, 4:3 ratio, display range : 0.1 unit <-> 100 units
    ProjectionMatrix = glm.perspective(FoV, 4.0 / 3.0, 0.1, 100.0)
    # Camera matrix
    ViewMatrix       = glm.lookAt(
                                position,           # Camera is here
                                position+direction, # and looks here : at the same position, plus "direction"
                                up                  # Head is up (set to 0,-1,0 to look upside-down)
                           )

    # For the next frame, the "last time" will be "now"
    lastTime = currentTime

def opengl_init():
    global window
    # Initialize the library
    if not glfw.init():
        print("Failed to initialize GLFW\n",file=sys.stderr)
        return False

    # Open Window and create its OpenGL context
    window = glfw.create_window(1024, 768, "Lab2", None, None) #(in the accompanying source code this variable will be global)
    glfw.window_hint(glfw.SAMPLES, 4)
    glfw.window_hint(glfw.CONTEXT_VERSION_MAJOR, 3)
    glfw.window_hint(glfw.CONTEXT_VERSION_MINOR, 3)
    glfw.window_hint(glfw.OPENGL_FORWARD_COMPAT, GL_TRUE)
    glfw.window_hint(glfw.OPENGL_PROFILE, glfw.OPENGL_CORE_PROFILE)

    if not window:
        print("Failed to open GLFW window. If you have an Intel GPU, they are not 3.3 compatible. Try the 2.1 version of the tutorials.\n",file=sys.stderr)
        glfw.terminate()
        return False

    # Initialize GLEW
    glfw.make_context_current(window)
    
    return True

def load_texture(file_name, texture_index):
    im = pil_open(file_name)
    width,height,image = im.size[0], im.size[1], im.convert('RGBA').tobytes('raw', 'RGBA', 0, -1)
    print('Reading Texture:',file_name,' Size',im.size)
    glActiveTexture(GL_TEXTURE0 + texture_index)

    texture = glGenTextures(1)
    glBindTexture(GL_TEXTURE_2D, texture)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, width, height, 0, GL_RGBA, GL_UNSIGNED_BYTE, image)

    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)

    largest_anisotropy = glGetFloatv(GL_MAX_TEXTURE_MAX_ANISOTROPY)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR_MIPMAP_LINEAR)
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_MAX_ANISOTROPY, largest_anisotropy)

    glGenerateMipmap(GL_TEXTURE_2D)

    return texture
def main():
# Initialize GLFW and open a window
    if not opengl_init():
        return
    rendering_mode='TEXTURED'
    # Enable key events
    glfw.set_input_mode(window,glfw.STICKY_KEYS,GL_TRUE) 
    glfw.set_input_mode(window, glfw.CURSOR , glfw.CURSOR_NORMAL)
    glfw.set_cursor_pos(window, 1024/2, 768/2)

    # Set opengl clear color to something other than red (color used by the fragment shader)
    glClearColor(1.0,1.0,1.0,0.0)
    
    # Enable depth test
    glEnable(GL_DEPTH_TEST)

    # Accept fragment if it closer to the camera than the former one
    glDepthFunc(GL_LESS)

    # Cull triangles which normal is not towards the camera
    glEnable(GL_CULL_FACE)

    vertex_array_id = glGenVertexArrays(1)
    glBindVertexArray( vertex_array_id )

    # Create and compile our GLSL program from the shaders
    if rendering_mode == 'TEXTURED':
        program_id = LoadShaders( "Explode.vs", "Explode.fs","Explode_ex_2.gs" )
    
    # Get a handle for our "MVP" uniform
    matrix_id = glGetUniformLocation(program_id, "MVP")
    view_matrix_id = glGetUniformLocation(program_id, "V")
    model_matrix_id = glGetUniformLocation(program_id, "M")
    # Get a handle for our "time' uniform
    time_id=glGetUniformLocation(program_id, "current_time")
    boom_id=glGetUniformLocation(program_id, "boom")

# Projection matrix : 45 Field of View, 4:3 ratio, display range : 0.1 unit <-> 100 units
    projection = glm.perspective(45.0, 4.0 / 3.0, 0.1, 100.0)
    
    # Camera matrix
    viewmatrix = glm.lookAt(glm.vec3(0.2,0,2), # Camera is at (0,0,0.5), in World Space
                    glm.vec3(0,0,0), # and looks at the origin
                    glm.vec3(0,1,0)) 
    
    # Model matrix : an identity matrix (model will be at the origin)
    modelmatrix = glm.mat4(1.0)
    modelmatrix =glm.translate(modelmatrix,glm.vec3(0.0, -0.1, 0.0))


    # Our ModelViewProjection : multiplication of our 3 matrices
    mvp = projection * viewmatrix * modelmatrix
    mvp=glm.rotate(mvp, -0.2,glm.vec3(0,1,0))
        
    # Load the texture
    if rendering_mode == 'TEXTURED':
        texture = load_texture("red.png",0)
        texture_id  = glGetUniformLocation(program_id, "diffuseMap")
    
    # Read our OBJ file
    vertices,faces,uvs,normals,colors = objloader.load("stanford_bunny_pbr.obj")
    vertex_data,uv_data,normal_data = objloader.process_obj( vertices,faces,uvs,normals,colors)
    print(len(vertex_data),len(uv_data),len(normal_data),vertex_data[0],uv_data[0],normal_data[0])
    # Our OBJ loader uses Python lists, convert to ctype arrays before sending to OpenGL
    vertex_data = objloader.generate_2d_ctypes(vertex_data)
    uv_data = objloader.generate_2d_ctypes(uv_data)
    normal_data = objloader.generate_2d_ctypes(normal_data)

    # Load OBJ in to a VBO
    vertex_buffer = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, vertex_buffer)
    glBufferData(GL_ARRAY_BUFFER, len(vertex_data) * 4 * 3, vertex_data, GL_STATIC_DRAW)

    uv_buffer = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, uv_buffer)
    glBufferData(GL_ARRAY_BUFFER, len(uv_data) * 4 * 2, uv_data, GL_STATIC_DRAW)

    normal_buffer = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, normal_buffer)
    glBufferData(GL_ARRAY_BUFFER, len(normal_data) * 4 * 3, normal_data, GL_STATIC_DRAW)

    # vsync and glfw do not play nice.  when vsync is enabled mouse movement is jittery.
    disable_vsyc()
    # Get a handle for our "LightPosition" uniform
    glUseProgram(program_id)
    light_id = glGetUniformLocation(program_id, "LightPosition_worldspace")

    last_time = glfw.get_time()
    frames = 0
    interact_mode=0
    boom=0
    glUniform1i(boom_id, boom)
    boom_time=glfw.get_time()
    while glfw.get_key(window,glfw.KEY_ESCAPE) != glfw.PRESS and not glfw.window_should_close(window):
        glClear(GL_COLOR_BUFFER_BIT| GL_DEPTH_BUFFER_BIT)

        current_time = glfw.get_time()
        if current_time - last_time >= 1.0:
            glfw.set_window_title(window,"Stanford Bunny.  FPS: %d"%(frames))
            frames = 0
            last_time = current_time

        glUseProgram(program_id)

        if glfw.get_key( window, glfw.KEY_I ) == glfw.PRESS :
            interact_mode=1
        if glfw.get_key( window, glfw.KEY_N ) == glfw.PRESS :
            interact_mode=0
        if glfw.get_key( window, glfw.KEY_B ) == glfw.PRESS :
            boom=1
            boom_time=glfw.get_time()
            # Set if boom is on
            glUniform1i(boom_id, boom)
        if interact_mode==1:    
            computeMatricesFromInputs(window)
            ProjectionMatrix = getProjectionMatrix();
            ViewMatrix = getViewMatrix();
            ModelMatrix = glm.mat4(1.0);
            glm.translate(ModelMatrix,glm.vec3(0.0, -0.1, 0.0))
            mvp = ProjectionMatrix * ViewMatrix * ModelMatrix;

        # Send our transformation to the currently bound shader, 
        # in the "MVP" uniform
        glUniformMatrix4fv(matrix_id, 1, GL_FALSE,glm.value_ptr(mvp))
        glUniformMatrix4fv(model_matrix_id, 1, GL_FALSE, glm.value_ptr(modelmatrix))
        glUniformMatrix4fv(view_matrix_id, 1, GL_FALSE, glm.value_ptr(viewmatrix))

        lightPos = glm.vec3(4,4,4)
        glUniform3f(light_id, lightPos.x, lightPos.y, lightPos.z)
        # Set the time
        if boom == 1 and (current_time-boom_time)<0.5:
            glUniform1f(time_id, 0)
        else:
            glUniform1f(time_id, current_time-boom_time)

        # Bind our texture in Texture Unit 0
        if rendering_mode == 'TEXTURED':
            glActiveTexture(GL_TEXTURE0);
            glBindTexture(GL_TEXTURE_2D, texture);
        # Set our "diffuseMap" sampler to user Texture Unit 0
            glUniform1i(texture_id, 0);
        #1rst attribute buffer : vertices
        glEnableVertexAttribArray(0)
        glBindBuffer(GL_ARRAY_BUFFER, vertex_buffer);
        glVertexAttribPointer(
            0,                  # attribute 0. No particular reason for 0, but must match the layout in the shader.
            3,                  # len(vertex_data)
            GL_FLOAT,           # type
            GL_FALSE,           # ormalized?
            0,                  # stride
            null                # array buffer offset (c_type == void*)
            )

         # 2nd attribute buffer : texture
        if rendering_mode == 'TEXTURED':
            glEnableVertexAttribArray(1)
            glBindBuffer(GL_ARRAY_BUFFER, uv_buffer);
            glVertexAttribPointer(
                1,                  # attribute 1. No particular reason for 1, but must match the layout in the shader.
                2,                  # len(vertex_data)
                GL_FLOAT,           # type
                GL_FALSE,           # ormalized?
                0,                  # stride
                null                # array buffer offset (c_type == void*)
                )

        # 3rd attribute buffer : normals
        glEnableVertexAttribArray(2);
        glBindBuffer(GL_ARRAY_BUFFER, normal_buffer);
        glVertexAttribPointer(
            2,                                  # attribute
            3,                                  # size
            GL_FLOAT,                           # type
            GL_FALSE,                           # ormalized?
            0,                                  # stride
            null                                # array buffer offset (c_type == void*)
        )
     # Draw the triangles, vertex data now contains individual vertices
        # so use array length
        glDrawArrays(GL_TRIANGLES, 0, len(vertex_data))

        # Not strictly necessary because we only have 
        glDisableVertexAttribArray(0)
        glDisableVertexAttribArray(1)
        glDisableVertexAttribArray(2)
    
    
        # Swap front and back buffers
        glfw.swap_buffers(window)

        # Poll for and process events
        glfw.poll_events()

        frames += 1
        time.sleep(0.1)
        # Swap front and back buffers
    # !Note braces around vertex_buffer and uv_buffer.  
    # glDeleteBuffers expects a list of buffers to delete
    glDeleteBuffers(1, [vertex_buffer])
    glDeleteBuffers(1, [uv_buffer])
    glDeleteBuffers(1, [normal_buffer])
    glDeleteProgram(program_id)
    if rendering_mode=='TEXTURED':
        glDeleteTextures([texture_id])
    glDeleteVertexArrays(1, [vertex_array_id])

    glfw.terminate()

if __name__ == "__main__":
    main()   

Compiling shader: Explode.vs
Compiling shader: Explode_ex_2.gs
Compiling shader: Explode.fs
Linking program
Reading Texture: red.png  Size (570, 518)
Model min values [-49582.778931, 33.529879, -37786.365274]
Model max values [49769.630432, 99382.652891, 37921.582469]
Model center of gravity [93.42575049999868, 49708.091385, 67.60859749999872]
17088 17088 17088 [-0.3156248706554078, 0.457451954653999, -0.4341356081804363] [0.697089, 0.396582] [0.688334, 0.579172, 0.436757]
